In [3]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer, IBMQ

c = ClassicalRegister(2)
q = QuantumRegister(5)

# Quantum registers
a = QuantumRegister(1, 'a')  # First input bit
b = QuantumRegister(1, 'b')  # Second input bit
cin = QuantumRegister(1, 'cin')  # Carry input bit
sum_out = QuantumRegister(1, 'sum_out')  # Sum output bit
cout = QuantumRegister(1, 'cout')  # Carry output bit
qc = QuantumCircuit(a, b, cin, sum_out, cout, c)

qc.initialize([0, 1], 0)  # Initialize 'a' to |0>
qc.initialize([0, 1], 1)  # Initialize 'b' to |0>
qc.initialize([1, 0], 2)  # Initialize 'cin' to |0>

# SUM Oracle
SUM = QuantumCircuit(4, name="SUM")
SUM.cx(0, 3)
SUM.cx(1, 3)
SUM.cx(2, 3)
SUM_instruction = SUM.to_instruction()
qc.append(SUM_instruction, [a[0], b[0], cin[0], sum_out[0]])
qc.barrier()

# Carry Oracle
CARRY = QuantumCircuit(5, name="CARRY")
CARRY.ccx(0, 1, 4)
CARRY.ccx(1, 2, 4)
CARRY.ccx(0, 2, 4)
CARRY_instruction = CARRY.to_instruction()
qc.append(CARRY_instruction, [a[0], b[0], cin[0], sum_out[0], cout[0]])

# Measurement
qc.measure(sum_out[0], c[1])
qc.measure(cout[0], c[0])

# Execution
provider = IBMQ.load_account()
backend = provider.get_backend('ibm_brisbane')

job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts(qc)
print(counts)


ibmqfactory.load_account:WARNING:2024-03-30 01:41:48,872: Credentials are already in use. The existing account in the session will be replaced.


{'10': 26, '01': 728, '11': 59, '00': 211}
